# 0.0 IMPORTS AND DATA LOADING

In [2]:
import findspark
import pandas as pd
from sklearn import metrics as m
findspark.init()
from pyspark.sql import SparkSession 
from pyspark.sql.functions import isnan, when, count, col, sum, expr, lit
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder, RobustScaler
from pyspark.ml.stat import Correlation
from pyspark.ml.classification import LogisticRegression

In [3]:
spark = SparkSession.builder.getOrCreate()

df = spark.read.csv('Fraud.csv',inferSchema=True,header=True)

df = df.drop(df.isFlaggedFraud)

# 1.0 DATA DESCRIPTION

In [4]:
df.printSchema()

root
 |-- step: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- nameOrig: string (nullable = true)
 |-- oldbalanceOrg: double (nullable = true)
 |-- newbalanceOrig: double (nullable = true)
 |-- nameDest: string (nullable = true)
 |-- oldbalanceDest: double (nullable = true)
 |-- newbalanceDest: double (nullable = true)
 |-- isFraud: integer (nullable = true)



In [5]:
df.show(5)

+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+-------+
|step|    type|  amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|isFraud|
+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+-------+
|   1| PAYMENT| 9839.64|C1231006815|     170136.0|     160296.36|M1979787155|           0.0|           0.0|      0|
|   1| PAYMENT| 1864.28|C1666544295|      21249.0|      19384.72|M2044282225|           0.0|           0.0|      0|
|   1|TRANSFER|   181.0|C1305486145|        181.0|           0.0| C553264065|           0.0|           0.0|      1|
|   1|CASH_OUT|   181.0| C840083671|        181.0|           0.0|  C38997010|       21182.0|           0.0|      1|
|   1| PAYMENT|11668.14|C2048537720|      41554.0|      29885.86|M1230701703|           0.0|           0.0|      0|
+----+--------+--------+-----------+-------------+--------------+-------

In [6]:
print(df.count())
print(len(df.columns))

6362620
10


In [7]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+----+----+------+--------+-------------+--------------+--------+--------------+--------------+-------+
|step|type|amount|nameOrig|oldbalanceOrg|newbalanceOrig|nameDest|oldbalanceDest|newbalanceDest|isFraud|
+----+----+------+--------+-------------+--------------+--------+--------------+--------------+-------+
|   0|   0|     0|       0|            0|             0|       0|             0|             0|      0|
+----+----+------+--------+-------------+--------------+--------+--------------+--------------+-------+



In [8]:
df.summary().toPandas()

,summary,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
0,count,6362620,6362620,6362620,6362620,6362620,6362620,6362620,6362620,6362620,6362620
1,mean,243.39724563151657,None,179861.90354913048,None,833883.1040744851,855113.6685785787,None,1100701.6665196505,1224996.3982019336,0.001290820448180152
2,stddev,142.3319710491305,None,603858.2314629437,None,2888242.6730375397,2924048.502954241,None,3399180.1129944543,3674128.9421196654,0.03590479680160438
3,min,1,CASH_IN,0.0,C1000000639,0.0,0.0,C1000004082,0.0,0.0,0
4,25%,156,None,13390.35,None,0.0,0.0,None,0.0,0.0,0
5,50%,239,None,74864.38,None,14211.23,0.0,None,132623.66,214605.81,0
6,75%,335,None,208722.05,None,107294.0,144232.62,None,942942.08,1111684.56,0
7,max,743,TRANSFER,9.244551664E7,C999999784,5.958504037E7,4.958504037E7,M999999784,3.5601588935E8,3.5617927892E8,1


# 2.0 EXPLORATORY DATA ANALYSIS

In [9]:
df.groupBy('isFraud').count().show()

+-------+-------+
|isFraud|  count|
+-------+-------+
|      1|   8213|
|      0|6354407|
+-------+-------+



In [10]:
df.groupBy('type').sum('isFraud').show()

+--------+------------+
|    type|sum(isFraud)|
+--------+------------+
|TRANSFER|        4097|
| CASH_IN|           0|
|CASH_OUT|        4116|
| PAYMENT|           0|
|   DEBIT|           0|
+--------+------------+



In [11]:
df.groupBy('type').count().show()

+--------+-------+
|    type|  count|
+--------+-------+
|TRANSFER| 532909|
| CASH_IN|1399284|
|CASH_OUT|2237500|
| PAYMENT|2151495|
|   DEBIT|  41432|
+--------+-------+



In [12]:
(df.filter((df.isFraud==1) & (df.oldbalanceDest==0) & (df.newbalanceDest==0) & (df.amount !=0) ).count())/(df.filter(df.isFraud==1).count())

0.4955558261293072

# 3.0 FILTERING DATA

In [14]:
df3 = df.drop(*['step','nameOrig','nameDest'])
df3 = df3.filter((df.type == 'TRANSFER') | (df.type == 'CASH_OUT') )

df3 = df3.filter(df.amount<20e6)
df3 = df3.filter(df.oldbalanceDest<5e7)
df3 = df3.filter(df.newbalanceDest<5e7)

In [15]:
df3.limit(5).toPandas()

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
0,TRANSFER,181.00,181.0,0.0,0.0,0.00,1
1,CASH_OUT,181.00,181.0,0.0,21182.0,0.00,1
2,CASH_OUT,229133.94,15325.0,0.0,5083.0,51513.44,0
3,TRANSFER,215310.30,705.0,0.0,22425.0,0.00,0
4,TRANSFER,311685.89,10835.0,0.0,6267.0,2719172.89,0


# 4.0 FEATURE ENGINEERING

In [16]:
df4 = df3.withColumn('errorBalanceOrig', (df3.oldbalanceOrg - df3.amount - df3.newbalanceOrig))

df4 = df4.withColumn('errorBalanceDest', (df4.oldbalanceDest + df4.amount - df4.newbalanceDest))

In [17]:
df4.limit(5).toPandas()

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,errorBalanceOrig,errorBalanceDest
0,TRANSFER,181.00,181.0,0.0,0.0,0.00,1,0.00,181.0
1,CASH_OUT,181.00,181.0,0.0,21182.0,0.00,1,0.00,21363.0
2,CASH_OUT,229133.94,15325.0,0.0,5083.0,51513.44,0,-213808.94,182703.5
3,TRANSFER,215310.30,705.0,0.0,22425.0,0.00,0,-214605.30,237735.3
4,TRANSFER,311685.89,10835.0,0.0,6267.0,2719172.89,0,-300850.89,-2401220.0


# 5.0 DATA PREPARATION

In [18]:
typeIndexer = StringIndexer(inputCol='type', outputCol='typeIndex')

df5 = typeIndexer.fit(df4).transform(df4)

one_hot_encoder = OneHotEncoder(inputCol='typeIndex', outputCol='typeOneHot')

# Use the encoder to transform the indexed dataframe
one_hot_fitted = one_hot_encoder.fit(df5)
df5 = one_hot_fitted.transform(df5)

In [19]:
df5.limit(5).toPandas()

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,errorBalanceOrig,errorBalanceDest,typeIndex,typeOneHot
0,TRANSFER,181.00,181.0,0.0,0.0,0.00,1,0.00,181.0,1.0,(0.0)
1,CASH_OUT,181.00,181.0,0.0,21182.0,0.00,1,0.00,21363.0,0.0,(1.0)
2,CASH_OUT,229133.94,15325.0,0.0,5083.0,51513.44,0,-213808.94,182703.5,0.0,(1.0)
3,TRANSFER,215310.30,705.0,0.0,22425.0,0.00,0,-214605.30,237735.3,1.0,(0.0)
4,TRANSFER,311685.89,10835.0,0.0,6267.0,2719172.89,0,-300850.89,-2401220.0,1.0,(0.0)


In [20]:
df5 = df5.drop('type','typeIndex')

In [45]:
train, test = df5.randomSplit([0.7,0.3],seed=42)

ones = train.filter(col("isFraud") == 1)
zeros = train.filter(col("isFraud") == 0)
n = 2
for i in range(n-1):
    ones = ones.union(ones)

In [46]:
zeros = zeros.limit(ones.count())
train = zeros.union(ones)

In [47]:
train.columns

['amount',
 'oldbalanceOrg',
 'newbalanceOrig',
 'oldbalanceDest',
 'newbalanceDest',
 'isFraud',
 'errorBalanceOrig',
 'errorBalanceDest',
 'typeOneHot']

In [48]:
assembler = VectorAssembler(inputCols=['amount','oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest',
                                       'errorBalanceOrig','errorBalanceDest','typeOneHot'], outputCol='features')

In [49]:
train = assembler.transform(train)
train = train.select(col('features'),col('isFraud'))

In [50]:
scaler = RobustScaler(inputCol="features", outputCol="scaledFeatures")
scalerFitted = scaler.fit(train)
train = scalerFitted.transform(train)

In [51]:
test = assembler.transform(test)
test = test.select(col('features'),col('isFraud'))

In [52]:
test = scalerFitted.transform(test)

# 6.0 MACHINE LEARNING MODELLING

In [53]:
lr = LogisticRegression(labelCol='isFraud')
lrn = lr.fit(train)
lr_pred = lrn.transform(test)

lr_pred = lr_pred.withColumn("label",col("isFraud").cast('double'))

In [54]:
lr_pred = lr_pred.toPandas()

In [70]:
#lr_pred['prediction'] = lr_pred['probability'].apply(lambda x: 1 if x[1]>0.5 else 0)

In [71]:
pd.crosstab(lr_pred['label'],lr_pred['prediction'])

prediction,0,1
label,,
0.0,683349,143581
1.0,289,2164


In [72]:
m.recall_score(lr_pred['label'],lr_pred['prediction'])

0.8821850794944965

In [73]:
m.precision_score(lr_pred['label'],lr_pred['prediction'])

0.014847850698137157

In [74]:
m.roc_auc_score(lr_pred['label'],lr_pred['prediction'])

0.8542768479716446